### Loading of CSV file

In [16]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_excel_loader import StructuredExcelLoader
import os
loader = CSVLoader(file_path='data/Excelfiles/Tsel Issue_Tracker_Updated13thNov_2025.xlsx' )
loader1=StructuredExcelLoader(file_path='data/Excelfiles/Tsel Issue_Tracker_Updated13thNov_2025.xlsx')
print(os.path.exists('data/Excelfiles/Tsel Issue_Tracker_Updated13thNov_2025.xlsx'))

#data = loader.load()

True


In [19]:
documents = loader1.load()
#documents1 = loader1.load()
print(f'Number of documents loaded: {len(documents)}')
for doc in documents:
    print(doc.page_content)
    print('---')
    print(doc.metadata)
#print(f'Number of documents1 loaded: {len(documents1)}')

Number of documents loaded: 2
SHEET: "Sheet1"

ROW 1:
CELL A1: Project
CELL B1: Date
CELL C1: Raised By
CELL D1: Topic
CELL E1: Issue
CELL F1: External/Internal
CELL G1: Action taken By
CELL H1: Status 
CELL I1: Resolution

ROW 2:
CELL A2: All Prod Servers
CELL B2: 04.12.2024
CELL C2: Surya Kant Sharma
CELL D2: Upgrade Failure 
CELL E2: There is some failure during upgrade in All 4 servers
CELL F2: Internal
CELL G2: Neeraj Mishra
CELL H2: Resolved
CELL I2: DCC Applied on all four server

ROW 3:
CELL A3: App04
CELL B3: 05.12.2024
CELL C3: Surya Kant Sharma
CELL D3: Failure in data load step
CELL E3: Tsel App04 :There is failure in 'error import-Discrepancies'
CELL F3: Internal
CELL G3: Surya Kant Sharma
CELL H3: Resolved
CELL I3: The feed is now completed post successful execution of the report job

ROW 4:
CELL A4: App02
CELL B4: 06.12.2024
CELL C4: Pranveer Singh
CELL D4: Failure in data load step
CELL E4: App02 - Import commit failed
CELL F4: Internal
CELL G4: Pranveer Singh
CELL H4: 

In [27]:
import pandas as pd

sheet_dict = pd.read_excel('data/Excelfiles/Tsel Issue_Tracker_Updated13thNov_2025.xlsx', sheet_name=None)

for sheet_name, df in sheet_dict.items():
    print(f"\nSheet: {sheet_name}")
    print(df.columns.tolist())


Sheet: Sheet1
['Project', 'Date', 'Raised By', 'Topic', 'Issue', 'External/Internal', 'Action taken By', 'Status ', 'Resolution']

Sheet: Sheet2
[]

Sheet: DCC Implementation
['Servers', 'Date of Implementation', 'DCC Name', 'Purpose/Issue']


In [2]:
from langchain_core.documents import Document
from typing import List
import pandas as pd

In [7]:
def safe(x):
    """Convert any value to clean string."""
    if pd.isna(x):
        return "N/A"
    return str(x).strip()

def load_excel(file_path: str) -> List[Document]:
    # Load ONLY Sheet1 (returns a DataFrame, not a dict)
    df = pd.read_excel(file_path, sheet_name="Sheet1")

    # Clean column names (remove trailing spaces)
    df.columns = df.columns.str.strip()

    documents = []

    for index, row in df.iterrows():
        content = (
            f"""The error occured in {safe(row['Project'])} and the actual topic is {safe(row['Topic'])}.
            The issue details is {safe(row['Issue'])} and the resolution is {safe(row['Resolution'])}."""
        )

        doc = Document(
            page_content=content,
            metadata={
                "source": file_path,
                "sheet_name": "Sheet1",
                "row_index": index
            }
        )

        documents.append(doc)

    return documents


# USE IT
excel_documents = load_excel('data/Excelfiles/Tsel Issue_Tracker_Updated13thNov_2025.xlsx')
print(f"Documents loaded: {len(excel_documents)}")
for doc in excel_documents[:10]:
    print(doc.page_content)
    print(doc.metadata)

Documents loaded: 204
The error occured in All Prod Servers and the actual topic is Upgrade Failure.
            The issue details is There is some failure during upgrade in All 4 servers and the resolution is DCC Applied on all four server.
{'source': 'data/Excelfiles/Tsel Issue_Tracker_Updated13thNov_2025.xlsx', 'sheet_name': 'Sheet1', 'row_index': 0}
The error occured in App04 and the actual topic is Failure in data load step.
            The issue details is Tsel App04 :There is failure in 'error import-Discrepancies' and the resolution is The feed is now completed post successful execution of the report job.
{'source': 'data/Excelfiles/Tsel Issue_Tracker_Updated13thNov_2025.xlsx', 'sheet_name': 'Sheet1', 'row_index': 1}
The error occured in App02 and the actual topic is Failure in data load step.
            The issue details is App02 - Import commit failed and the resolution is increasing undotbs size and nexted the feed.
{'source': 'data/Excelfiles/Tsel Issue_Tracker_Updated13th